In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data.csv')

In [7]:
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
df.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [18]:
from sklearn.model_selection import train_test_split 
X = df.drop(['diagnosis'],axis=1)
y = df['diagnosis']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [31]:
from sklearn.ensemble import StackingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 

base_models = [
    ('tree', DecisionTreeClassifier(max_depth=3)),
    ('SVM', SVC(kernel='rbf')),
    ('NB', GaussianNB()),
]

meta_model = LogisticRegression()

stack = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

In [32]:
stack.fit(X_train,y_train)

StackingClassifier(cv=5,
                   estimators=[('tree', DecisionTreeClassifier(max_depth=3)),
                               ('SVM', SVC()), ('NB', GaussianNB())],
                   final_estimator=LogisticRegression())

In [33]:
y_pred = stack.predict(X_test)

In [34]:
from sklearn.metrics import classification_report 
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           B       0.91      0.98      0.94        99
           M       0.97      0.86      0.91        72

    accuracy                           0.93       171
   macro avg       0.94      0.92      0.93       171
weighted avg       0.93      0.93      0.93       171



In [35]:
for name,model in base_models:
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(f"{name}:classification report : {classification_report(y_test,y_pred)}")

tree:classification report :               precision    recall  f1-score   support

           B       0.91      0.98      0.94        99
           M       0.97      0.86      0.91        72

    accuracy                           0.93       171
   macro avg       0.94      0.92      0.93       171
weighted avg       0.93      0.93      0.93       171

SVM:classification report :               precision    recall  f1-score   support

           B       0.82      0.99      0.90        99
           M       0.98      0.71      0.82        72

    accuracy                           0.87       171
   macro avg       0.90      0.85      0.86       171
weighted avg       0.89      0.87      0.87       171

NB:classification report :               precision    recall  f1-score   support

           B       0.89      0.98      0.93        99
           M       0.97      0.83      0.90        72

    accuracy                           0.92       171
   macro avg       0.93      0.91      0.91 

In [52]:
from sklearn.ensemble import RandomForestClassifier
super_model = RandomForestClassifier(n_estimators=100,max_depth=6)
super_model.fit(X_train,y_train)
y_pred = super_model.predict(X_test)

In [53]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           B       0.93      0.98      0.96        99
           M       0.97      0.90      0.94        72

    accuracy                           0.95       171
   macro avg       0.95      0.94      0.95       171
weighted avg       0.95      0.95      0.95       171



RandomForest demonstrated strong performance !